# Transliteration from English to Marathi
## Using Encoder - Decoder with Attention Layers

### Team: (BCA 1 - Group 2)
1. Manas Acharya (01, 172066)
2. Ritika Bhole (07, 172089)
3. Sahil Nirkhe (12, 172099)
4. Sanket Dalvi (13, 172104) 

## Acquire Dataset

In [ ]:
!wget https://storage.googleapis.com/gresearch/dakshina/dakshina_dataset_v1.0.tar

--2021-05-16 10:17:18--  https://storage.googleapis.com/gresearch/dakshina/dakshina_dataset_v1.0.tar
Resolving storage.googleapis.com (storage.googleapis.com)... 172.253.122.128, 172.253.63.128, 142.250.31.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.253.122.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2008340480 (1.9G) [application/x-tar]
Saving to: ‘dakshina_dataset_v1.0.tar’

dakshina_dataset_v1 100%[===================>]   1.87G  34.4MB/s    in 17s     

2021-05-16 10:17:35 (113 MB/s) - ‘dakshina_dataset_v1.0.tar’ saved [2008340480/2008340480]



In [ ]:
!tar -xf dakshina_dataset_v1.0.tar

In [ ]:
!head /content/dakshina_dataset_v1.0/mr/romanized/mr.romanized.rejoined.test.roman.txt

swar
19 vya shatakamadhye jevha British East India Companyne Bharatacha sampurna prashaskiya karbhar aplya hatat ghetla tevha tya prakriyemadhye ek navin asa nokardar ingraji bolnara madhyam varga udayaas aala.
Mandirawar shikharaaivaji ekavar ek ashi Shivlinge ahet.
chal dhar pakad
sadharan teen varshantun ekda ha nisarga chamatkar pahayla milto.
sakshtva mhanaje pahane athava janane.
hajaro varshanpasun mahaan vyaktinchya murti ghadavlya jaat ahet.
tyanna mahit hote ki Munich akramankartyanchya sutkesathi Germanyla dahashatvadi karyancha saamna karava lagel.
cheda lahana asalyamule vyayama va saundryachya drishtikonatun uttam asate.
Kanitkar yanni anek aitihasik pustakanche lekhan kele hote.


In [ ]:
!head /content/dakshina_dataset_v1.0/mr/romanized/mr.romanized.rejoined.test.native.txt

स्वर
१९ व्या शतकामध्ये जेव्हा ब्रिटिश ईस्ट इंडिया कंपनीने भारताचा संपूर्ण प्रशासकीय कारभार आपल्या हातात घेतला तेव्हा त्या प्रक्रियेमध्ये एक नवीन असा नोकरदार इंग्रजी बोलणारा मध्यम वर्ग उदयास आला.
मंदिरावर शिखराऐवजी एकावर एक अशी शिवलिंगे आहेत.
चल धर पकड
साधारण तीन वर्षांतून एकदा हा निसर्ग चमत्कार पहायला मिळतो.
साक्षत्व म्हणजे पाहणे अथवा जाणणे.
हजारो वर्षांपासून महान व्यक्तींच्या मूर्ती घडवल्या जात आहेत.
त्यांना माहीत होते की म्युनिच आक्रमणकर्त्यांच्या सुटकेसाठी जर्मनीला दहशतवादी कार्यांचा सामना करावा लागेल.
छेद लहान असल्यामुळे व्यायाम व सौंदर्याच्या दृष्टिकोनातून उत्तम असते.
कानिटकर यांनी अनेक ऐतिहासिक पुस्तकांचे लेखन केले होते.


## Importing libraries

In [ ]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf
print(tf.__version__)
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

import os
import io
import time
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import unicodedata
import re
import time
from pprint import pprint

2.4.1


## Text Processing helper functions

### Devnagiri alphabets list

In [ ]:
marathi_alphabets = [chr(alpha) for alpha in range(2304, 2432)]
marathi_alphabet_size = len(marathi_alphabets)

marathi_alpha2index = {'<start>': 0,'<end>' : 1}
for index, alpha in enumerate(marathi_alphabets):
    marathi_alpha2index[alpha] = index+1

# pprint(marathi_alpha2index)

{'<end>': 1,
 '<start>': 0,
 'ऀ': 1,
 'ँ': 2,
 'ं': 3,
 'ः': 4,
 'ऄ': 5,
 'अ': 6,
 'आ': 7,
 'इ': 8,
 'ई': 9,
 'उ': 10,
 'ऊ': 11,
 'ऋ': 12,
 'ऌ': 13,
 'ऍ': 14,
 'ऎ': 15,
 'ए': 16,
 'ऐ': 17,
 'ऑ': 18,
 'ऒ': 19,
 'ओ': 20,
 'औ': 21,
 'क': 22,
 'ख': 23,
 'ग': 24,
 'घ': 25,
 'ङ': 26,
 'च': 27,
 'छ': 28,
 'ज': 29,
 'झ': 30,
 'ञ': 31,
 'ट': 32,
 'ठ': 33,
 'ड': 34,
 'ढ': 35,
 'ण': 36,
 'त': 37,
 'थ': 38,
 'द': 39,
 'ध': 40,
 'न': 41,
 'ऩ': 42,
 'प': 43,
 'फ': 44,
 'ब': 45,
 'भ': 46,
 'म': 47,
 'य': 48,
 'र': 49,
 'ऱ': 50,
 'ल': 51,
 'ळ': 52,
 'ऴ': 53,
 'व': 54,
 'श': 55,
 'ष': 56,
 'स': 57,
 'ह': 58,
 'ऺ': 59,
 'ऻ': 60,
 '़': 61,
 'ऽ': 62,
 'ा': 63,
 'ि': 64,
 'ी': 65,
 'ु': 66,
 'ू': 67,
 'ृ': 68,
 'ॄ': 69,
 'ॅ': 70,
 'ॆ': 71,
 'े': 72,
 'ै': 73,
 'ॉ': 74,
 'ॊ': 75,
 'ो': 76,
 'ौ': 77,
 '्': 78,
 'ॎ': 79,
 'ॏ': 80,
 'ॐ': 81,
 '॑': 82,
 '॒': 83,
 '॓': 84,
 '॔': 85,
 'ॕ': 86,
 'ॖ': 87,
 'ॗ': 88,
 'क़': 89,
 'ख़': 90,
 'ग़': 91,
 'ज़': 92,
 'ड़': 93,
 'ढ़': 94,
 'फ़': 95,
 'य़': 96,
 'ॠ': 97,
 'ॡ': 98,
 

### Preprocessing functions

In [ ]:
def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s))
        # if unicodedata.category(c) != 'Mn')


def preprocess_sentence_english(w):
    w = unicode_to_ascii(w.lower().strip())
    w = w.replace('-', ' ').replace(',', ' ')
    
    # creating a space between a word and the punctuation following it
    # eg: "he is a boy." => "he is a boy ." 
    # Reference:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
    w = re.sub(r"([?.!,¿])", r" \1 ", w)
    w = re.sub(r'[" "]+', " ", w)
    
    # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
    w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)
    
    w = w.rstrip().strip()
    
    # adding a start and an end token to the sentence
    # so that the model know when to start and stop predicting.
    w = '@' + w + '#'
    return w.split()

def preprocess_sentence_marathi(w):
    w = unicode_to_ascii(w.strip())
    w = w.replace('-', ' ').replace(',', ' ')
    cleaned_line = ''
    for char in w:
        if char in marathi_alpha2index or char == ' ':
            cleaned_line += char
    cleaned_line = cleaned_line.rstrip().strip()
    
    # adding a start and an end token to the sentence
    # so that the model know when to start and stop predicting.
    cleaned_line = '@' + cleaned_line + '#'
    return cleaned_line.split()

### Creating datasets

In [ ]:
def create_dataset(data):
    lang1_words = []
    lang2_words = []

    for index, row in data.iterrows():
        try:
            wordlist1 = preprocess_sentence_english(row['en']) # clean English words.
            wordlist2 = preprocess_sentence_marathi(row['mr']) # clean marathi words.
            if len(wordlist1) != len(wordlist2):
                print('Skipping: ', row['en'], ' - ', row['mr'])
                continue
            for word in wordlist1:
                lang1_words.append(word)
            for word in wordlist2:
                lang2_words.append(word)
        except Exception as e:
            print(f"Found error at {row}")
    return [lang1_words,lang2_words]

#### Train dataset

In [ ]:
train = pd.read_csv("/content/dakshina_dataset_v1.0/mr/lexicons/mr.translit.sampled.train.tsv", sep="\t", names = ['mr', 'en', 'lex'], header=None)

In [ ]:
train.dropna(inplace=True)

In [ ]:
train_data = create_dataset(train)

In [ ]:
for i in range(50):
    print(f"{train_data[0][-i]} => {train_data[1][-i]}")

@angry# => @अँग्री#
@yalan# => @ॲलन#
@elan# => @ॲलन#
@allan# => @ॲलन#
@alana# => @ॲलन#
@alan# => @ॲलन#
@ailan# => @ॲलन#
@hrudayat# => @ह्रदयात#
@hyuustana# => @ह्यूस्टन#
@hyustana# => @ह्यूस्टन#
@huustun# => @ह्यूस्टन#
@huusetun# => @ह्यूस्टन#
@hustun# => @ह्यूस्टन#
@houuston# => @ह्यूस्टन#
@houuseton# => @ह्यूस्टन#
@houston# => @ह्यूस्टन#
@houseton# => @ह्यूस्टन#
@hoston# => @ह्यूस्टन#
@hayauuston# => @ह्यूस्टन#
@hayauustan# => @ह्यूस्टन#
@hayaustun# => @ह्यूस्टन#
@hayauston# => @ह्यूस्टन#
@hayaustan# => @ह्यूस्टन#
@haustan# => @ह्यूस्टन#
@hausetan# => @ह्यूस्टन#
@hastan# => @ह्यूस्टन#
@hyooman# => @ह्यूमन#
@human# => @ह्यूमन#
@hyustan# => @ह्युस्टन#
@houston# => @ह्युस्टन#
@hyasarakhya# => @ह्यासारख्या#
@hyasarakha# => @ह्यासारख्या#
@hyasathi# => @ह्यासाठी#
@hyasathee# => @ह्यासाठी#
@hyas# => @ह्यास#
@hyavarun# => @ह्यावरून#
@hyavaroon# => @ह्यावरून#
@hyavarun# => @ह्यावरुन#
@hyavaroon# => @ह्यावरुन#
@hyavar# => @ह्यावर#
@hyala# => @ह्याला#
@hyamule# => @ह्यामुळे#
@hyamage# => @ह्याम

In [ ]:
len(train_data[0])

56300

#### Validation dataset

In [ ]:
dev = pd.read_csv("/content/dakshina_dataset_v1.0/mr/lexicons/mr.translit.sampled.dev.tsv", sep="\t", names = ['mr', 'en', 'lex'], header=None)

In [ ]:
dev.dropna(inplace=True)

In [ ]:
dev_data = create_dataset(dev)

In [ ]:
for i in range(10):
    print(f"{dev_data[0][i]} => {dev_data[1][i]}")

@aendarsanla# => @अँडरसनला#
@andersonla# => @अँडरसनला#
@andrew# => @अँड्र्यू#
@ankaleekar# => @अंकलीकर#
@ankalikar# => @अंकलीकर#
@ankalivar# => @अंकलीकर#
@anchal# => @अंचल#
@antarctica# => @अंटार्क्टिका#
@antarctika# => @अंटार्क्टिका#
@antarctiqa# => @अंटार्क्टिका#


In [ ]:
len(dev_data[0])

5658

## Vocabulary Class

In [ ]:
class WordIndex():
  def __init__(self, lang):
    self.lang = lang
    self.word2idx = {}
    self.idx2word = {}
    self.vocab = set()
    
    self.create_index()
    
  def create_index(self):
    for phrase in self.lang:
      for l in phrase:
        self.vocab.update(l)
    
    self.vocab = sorted(self.vocab)
    
    self.word2idx['<pad>'] = 0
    for index, word in enumerate(self.vocab):
      self.word2idx[word] = index + 1
    
    for word, index in self.word2idx.items():
      self.idx2word[index] = word

In [ ]:
def max_length(tensor):
    return max(len(t) for t in tensor)


def load_dataset(pairs, max_length_inp=None, max_length_tar=None):
    # index language using the class defined above    
    inp_lang = WordIndex(pairs[0])
    targ_lang = WordIndex(pairs[1])
    
    # Vectorize the input and target languages
    # English words
    input_tensor = [[inp_lang.word2idx[s] for s in en] for en in pairs[0]]
    
    # marathi words
    target_tensor = [[targ_lang.word2idx[s] for s in mr] for mr in pairs[1]]
    
    # Calculate max_length of input and output tensor
    # Here, we'll set those to the longest sentence in the dataset
    if max_length_inp is None or max_length_tar is None:
        max_length_inp, max_length_tar = max_length(input_tensor), max_length(target_tensor)
    
    # Padding the input and output tensor to the maximum length
    input_tensor = tf.keras.preprocessing.sequence.pad_sequences(input_tensor, 
                                                                 maxlen=max_length_inp,
                                                                 padding='post')
    
    target_tensor = tf.keras.preprocessing.sequence.pad_sequences(target_tensor, 
                                                                  maxlen=max_length_tar, 
                                                                  padding='post')
    
    return input_tensor, target_tensor, inp_lang, targ_lang, max_length_inp, max_length_tar

In [ ]:
def convert(lang, tensor):
  for t in tensor:
    if t!=0:
      print ("%d ----> %s" % (t, lang.idx2word[t]))

## Model Building

### Train and val data

In [ ]:
input_tensor_train, target_tensor_train, inp_lang, targ_lang, max_length_inp, max_length_tar = load_dataset(train_data)

In [ ]:
input_tensor_val, target_tensor_val, inp_lang, targ_lang, max_length_inp, max_length_tar = load_dataset(dev_data, max_length_inp, max_length_tar)

In [ ]:
len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val)

(56300, 56300, 5658, 5658)

Splits

In [ ]:
input_tensor, target_tensor, inp_lang, targ_lang, max_length_inp, max_length_tar = load_dataset(train_data)

In [ ]:
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.05)

In [ ]:
len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val)

(53485, 53485, 2815, 2815)

### Hyperparameters

In [ ]:
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 128
steps_per_epoch = len(input_tensor_train)//BATCH_SIZE
embedding_dim = 256
units = 1024
vocab_inp_size = len(inp_lang.word2idx)+1
vocab_tar_size = len(targ_lang.word2idx)+1

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [ ]:
example_input_batch, example_target_batch = next(iter(dataset))
example_input_batch.shape, example_target_batch.shape

(TensorShape([128, 25]), TensorShape([128, 22]))

### Encoder - Decoder with Attention

In [ ]:
class Encoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
    super(Encoder, self).__init__()
    self.batch_sz = batch_sz
    self.enc_units = enc_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.enc_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')

  def call(self, x, hidden):
    x = self.embedding(x)
    output, state = self.gru(x, initial_state = hidden)
    return output, state

  def initialize_hidden_state(self):
    return tf.zeros((self.batch_sz, self.enc_units))

In [ ]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)

# sample input
sample_hidden = encoder.initialize_hidden_state()
sample_output, sample_hidden = encoder(example_input_batch, sample_hidden)
print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
print ('Encoder Hidden state shape: (batch size, units) {}'.format(sample_hidden.shape))

Encoder output shape: (batch size, sequence length, units) (128, 25, 1024)
Encoder Hidden state shape: (batch size, units) (128, 1024)


In [ ]:
class Attention(tf.keras.Model):
  def __init__(self, units):
    super(Attention, self).__init__()
    self.W1 = tf.keras.layers.Dense(units)
    self.W2 = tf.keras.layers.Dense(units)
    self.V = tf.keras.layers.Dense(1)

  def call(self, query, values):
    # hidden shape == (batch_size, hidden size)
    # hidden_with_time_axis shape == (batch_size, 1, hidden size)
    # we are doing this to perform addition to calculate the score
    hidden_with_time_axis = tf.expand_dims(query, 1)

    # score shape == (batch_size, max_length, 1)
    # we get 1 at the last axis because we are applying score to self.V
    # the shape of the tensor before applying self.V is (batch_size, max_length, units)
    score = self.V(tf.nn.tanh(
        self.W1(values) + self.W2(hidden_with_time_axis)))

    # attention_weights shape == (batch_size, max_length, 1)
    attention_weights = tf.nn.softmax(score, axis=1)

    # context_vector shape after sum == (batch_size, hidden_size)
    context_vector = attention_weights * values
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights

In [ ]:
attention_layer = Attention(10)
attention_result, attention_weights = attention_layer(sample_hidden, sample_output)

print("Attention result shape: (batch size, units) {}".format(attention_result.shape))
print("Attention weights shape: (batch_size, sequence_length, 1) {}".format(attention_weights.shape))

Attention result shape: (batch size, units) (128, 1024)
Attention weights shape: (batch_size, sequence_length, 1) (128, 25, 1)


In [ ]:
class Decoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
    super(Decoder, self).__init__()
    self.batch_sz = batch_sz
    self.dec_units = dec_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.dec_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    self.fc = tf.keras.layers.Dense(vocab_size)

    # used for attention
    self.attention = Attention(self.dec_units)

  def call(self, x, hidden, enc_output):
    # enc_output shape == (batch_size, max_length, hidden_size)
    context_vector, attention_weights = self.attention(hidden, enc_output)


    # x shape after passing through embedding == (batch_size, 1, embedding_dim)
    x = self.embedding(x)

    # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
    x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

    # passing the concatenated vector to the GRU
    output, state = self.gru(x)

    # output shape == (batch_size * 1, hidden_size)
    output = tf.reshape(output, (-1, output.shape[2]))

    # output shape == (batch_size, vocab)
    x = self.fc(output)

    return x, state, attention_weights

In [ ]:
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)

sample_decoder_output, _, _ = decoder(tf.random.uniform((128, 1)),
                                      sample_hidden, sample_output)

print ('Decoder output shape: (batch_size, vocab size) {}'.format(sample_decoder_output.shape))

Decoder output shape: (batch_size, vocab size) (128, 68)


In [ ]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)

In [ ]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

## Training

In [ ]:
@tf.function
def train_step(inp, targ, enc_hidden):
  loss = 0

  with tf.GradientTape() as tape:
    enc_output, enc_hidden = encoder(inp, enc_hidden)
    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([targ_lang.word2idx['@']] * BATCH_SIZE, 1)

    # Teacher forcing - feeding the target as the next input
    for t in range(1, targ.shape[1]):
      # passing enc_output to the decoder
      predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)
      loss += loss_function(targ[:, t], predictions)
      # using teacher forcing
      dec_input = tf.expand_dims(targ[:, t], 1)

  batch_loss = (loss / int(targ.shape[1]))
  variables = encoder.trainable_variables + decoder.trainable_variables
  gradients = tape.gradient(loss, variables)
  optimizer.apply_gradients(zip(gradients, variables))

  return batch_loss

In [ ]:
tf.test.is_gpu_available()

True

In [ ]:
EPOCHS = 10

for epoch in range(EPOCHS):
  start = time.time()

  enc_hidden = encoder.initialize_hidden_state()
  total_loss = 0

  for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
    batch_loss = train_step(inp, targ, enc_hidden)
    total_loss += batch_loss

    if batch % 100 == 0:
        print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                     batch,
                                                     batch_loss.numpy()))
  # saving (checkpoint) the model every 2 epochs
  if (epoch + 1) % 2 == 0:
    checkpoint.save(file_prefix = checkpoint_prefix)

  print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                      total_loss / steps_per_epoch))
  print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

Epoch 1 Batch 0 Loss 1.5387
Epoch 1 Batch 100 Loss 0.9606
Epoch 1 Batch 200 Loss 0.4905
Epoch 1 Batch 300 Loss 0.2433
Epoch 1 Batch 400 Loss 0.2102
Epoch 1 Loss 0.6020
Time taken for 1 epoch 121.74392604827881 sec

Epoch 2 Batch 0 Loss 0.2115
Epoch 2 Batch 100 Loss 0.1817
Epoch 2 Batch 200 Loss 0.1676
Epoch 2 Batch 300 Loss 0.1430
Epoch 2 Batch 400 Loss 0.1513
Epoch 2 Loss 0.1639
Time taken for 1 epoch 98.15489220619202 sec

Epoch 3 Batch 0 Loss 0.1388
Epoch 3 Batch 100 Loss 0.1227
Epoch 3 Batch 200 Loss 0.1073
Epoch 3 Batch 300 Loss 0.1036
Epoch 3 Batch 400 Loss 0.1016
Epoch 3 Loss 0.1175
Time taken for 1 epoch 97.7836012840271 sec

Epoch 4 Batch 0 Loss 0.1192
Epoch 4 Batch 100 Loss 0.0950
Epoch 4 Batch 200 Loss 0.1280
Epoch 4 Batch 300 Loss 0.1045
Epoch 4 Batch 400 Loss 0.1074
Epoch 4 Loss 0.1007
Time taken for 1 epoch 97.88889980316162 sec

Epoch 5 Batch 0 Loss 0.0833
Epoch 5 Batch 100 Loss 0.0823
Epoch 5 Batch 200 Loss 0.0788
Epoch 5 Batch 300 Loss 0.0736
Epoch 5 Batch 400 Loss 0.0

## Using the Predictions

In [ ]:
def evaluate(sentence):
    attention_plot = np.zeros((max_length_tar, max_length_inp))

    sentence = preprocess_sentence_english(sentence)[0]

    inputs = [inp_lang.word2idx[i] for i in sentence]
    inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                           maxlen=max_length_inp,
                                                           padding='post')
    inputs = tf.convert_to_tensor(inputs)
    result = ''
    hidden = [tf.zeros((1, units))]
    enc_out, enc_hidden = encoder(inputs, hidden)
    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([targ_lang.word2idx['@']], 0)

    options = []
    for t in range(max_length_tar):
        predictions, dec_hidden, attention_weights = decoder(dec_input,
                                                             dec_hidden,
                                                             enc_out)

        # storing the attention weights to plot later on
        attention_weights = tf.reshape(attention_weights, (-1, ))
        attention_plot[t] = attention_weights.numpy()

        options_for_curr_char = []
        for idx, i in enumerate(np.argsort(predictions[0])[::-1][:2]):
            if idx >0 and predictions[0][i] > 10:
                options_for_curr_char.append(targ_lang.idx2word[i])
            elif idx==0:
                options_for_curr_char.append(targ_lang.idx2word[i])
            # print(f"{targ_lang.idx2word[i]} ({predictions[0][i]})", end = " ")
        options.append(options_for_curr_char)
        # print()
        predicted_id = tf.argmax(predictions[0]).numpy()
        result += targ_lang.idx2word[predicted_id] + ' '
        if targ_lang.idx2word[predicted_id] == '#':
            return options, result, sentence, attention_plot

        # the predicted ID is fed back into the model
        dec_input = tf.expand_dims([predicted_id], 0)

    return options, result, sentence, attention_plot

def plot_attention(attention, sentence, predicted_sentence):
#     print(predicted_sentence)
    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(1, 1, 1)
    ax.matshow(attention, cmap='viridis')

    fontdict = {'fontsize': 14}

    ax.set_xticklabels([''] + sentence, fontdict=fontdict, rotation=90)
    ax.set_yticklabels([''] + predicted_sentence, fontdict=fontdict)

    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.show()

In [ ]:
def transliterate(sentence):
    options, result, sentence, attention_plot = evaluate(sentence)
    return ''.join(result.split(' '))

In [ ]:
def transliterate_sentence(sentence):
    translated = ""
    for i in sentence.split(" "):
        translated += " " + transliterate(i)[:-1]
    print(translated)

In [ ]:
transliterate_sentence("maala don dole aahet")

[['म'], ['ा'], ['ल', 'ळ'], ['ा'], ['#']]
[['ड', 'द'], ['ो', 'ॉ'], ['न'], ['#']]
[['ड'], ['ो'], ['ळ', 'ल'], ['े', '#'], ['#']]
[['आ', 'अ'], ['ह'], ['े', 'ा'], ['त', 'ट'], ['#', 'े']]
 माला डोन डोळे आहेत


In [ ]:
transliterate("dole")

[['ड'], ['ो'], ['ळ', 'ल'], ['े', '#'], ['#']]


'डोळे#'

In [ ]:
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))